# Machine learning - using LASSO to select features.

 This is one of my training projects. The main task is to investigate tuning parameter impact on RSS and magnitude of coefficients using wide range of lambda values. Task idea and dataset come from coursera's machine learning course (one of the quiz sections).

In [1]:
import numpy as np
from sklearn import linear_model
import pandas as pd
from pythonds.basic.stack import Stack

## Data

Loading the sales dataset using pandas. Setting up column data types at the beginning may avoid memory problems with huge datasets (dtype determination of each column).

In [2]:
dtype_dict = {'bathrooms': float, 'waterfront': int, 'sqft_above': int, 'sqft_living15': float, 'grade': int,
              'yr_renovated': int, 'price': float, 'bedrooms': float, 'zipcode': str, 'long': float, 
              'sqft_lot15': float, 'sqft_living': float, 'floors': float, 'condition': int, 'lat': float, 'date': str,
              'sqft_basement': int, 'yr_built': int, 'id': str, 'sqft_lot': int, 'view': int}

In [4]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

Transformating existing inputs to get new features.

In [5]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(np.sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(np.sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [6]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated'
                ]

Learning regression weights using large L1 penalty value - 500.

In [7]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True)
model_all.fit(sales[all_features], sales['price'])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Lasso approach brings sparse solutions. Most of the weights were excluded from the model when we put large L1 penalty value.

In [30]:
print('Number of inputs: {}'.format(len(all_features)))
print(model_all.coef_[model_all.coef_ != 0])

Number of inputs: 17
[   134.43931396  24750.00458561  61749.10309071]


## Importing splitted data sets

Getting already splitted up data into testing, training, validation sets.

In [9]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [10]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(np.sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(np.sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(np.sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(np.sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(np.sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(np.sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

## Selecting L1 penalty

Creating an array with L1 penalty values for further impact exploration.

In [12]:
l1_penalty = np.logspace(1, 7, num=13)

Now I'm going to use prepared training and validation sets to learn new models with 13 different L1 penalties. Loop puts key-value pairs of L1 penalty and equivalent RSS value for this specific parameter to 'rss_dict'. The goal is to define penalty value where we get the lowest RSS in total cost.

In [13]:
rss_dict = {}
for l1 in l1_penalty:
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    prediction = model.predict(validation[all_features])
    rss = sum((prediction - validation['price']) ** 2)
    rss_dict[l1] = rss

In [14]:
l1_optimum = list(rss_dict.keys())[list(rss_dict.values()).index(min(rss_dict.values()))]

L1 penalty value that produced the lowest RSS error was 10 (the lowest one from penalties set).

In [33]:
l1_optimum

10.0

Computing the RSS on test data with the best L1 penalty.

In [34]:
model_test = linear_model.Lasso(alpha=l1_optimum, normalize=True)
model_test.fit(training[all_features], training['price'])
prediction = model_test.predict(testing[all_features])
rss_test = sum((prediction - testing['price']) ** 2)

Counting number of nonzero weights for the best L1 penalty including intercept weight.

In [17]:
nonzero_weights = np.count_nonzero(model_test.coef_) + np.count_nonzero(model_test.intercept_)

## Exploring some range of L1 penalty values to get desired sparsity

Task - limit a model to 7 nonzero weights of features and find a narrow region of L1 penalty values where models are likely to have desired number of weights. Find both ends of narrow range of L1 penalty.

It can be done with L1 penalty values generator working in certain range. When penalty value hits number of nonzero weights of the model equal to 7, min_l1 variable is being changed to actual penalty value. When model with some penalty value hits number of nonzero weights larger than desired we get previous penalty value using Stack() methods. The while loop contains 2 more scenarios where there is no such desired L1 penalty or we found only one end of narrow region.

In [18]:
def l1_generator(start, stop, step=1):
    return (x for x in range(start, stop+1, step))

In [19]:
starting_val = 100
stopping_val = 300
step_val = 1

gen = l1_generator(starting_val, stopping_val, step_val)

In [20]:
stack = Stack()

min_l1 = None
max_l2 = None

max_non_zeros = 7

In [22]:
while True:
    model_gen = linear_model.Lasso(alpha=next(gen), normalize=True)
    model_gen.fit(training[all_features], training['price'])
    # Checking if # of nonzero coefficients is equal to max_non_zeros value.    
    x = (np.count_nonzero(model_gen.coef_) == max_non_zeros)
    
    # Getting the head of our range.
    if x and min_l1 is None:
        min_l1 = model_gen.get_params()['alpha']
        stack.push(min_l1)
    # Getting the tail of our range.
    elif min_l1 is not None and x is False:
        popped_val = stack.pop()
        max_l2 = popped_val
        break
    # If the last L1 parameter is equal to max_non_zero scenario
    elif model_gen.get_params()['alpha'] == stopping_val and x:
        max_l2 = model_gen.get_params()['alpha']
        break
    # No such tuning parameters in the list scenario.
    elif model_gen.get_params()['alpha'] == stopping_val:
        print('There is no L1 penalty value for this specific criteria.')
        break
    else:
        stack.push(model_gen.get_params()['alpha'])

In [23]:
min_l1, max_l2

(139, 153)

## Exploring the narrow range of penalties to find a solution with the lowest RSS

Let's now explore region we found. We look for L1 penalty value in this range that minimizes the RSS on the validation set.

In [24]:
narrow_val = np.linspace(min_l1, max_l2, 20)

In [25]:
rss_valid = {}

for l1_penalties in narrow_val:
    model_narr = linear_model.Lasso(alpha=l1_penalties, normalize=True)
    model_narr.fit(training[all_features], training['price'])
    prediction = model_narr.predict(validation[all_features])
    rss = sum((prediction - validation['price']) ** 2)
    rss_valid[l1_penalties] = rss

In [32]:
l1_opt = list(rss_valid.keys())[list(rss_valid.values()).index(min(rss_valid.values()))]
l1_opt

139.0

## Results

Creating a model with 7 non-zero coefficients using L1 penalty with the lowest RSS.

In [27]:
model_final = linear_model.Lasso(alpha=l1_opt, normalize=True)
model_final.fit(training[all_features], training['price'])
final_non_zero = model_final.coef_[model_final.coef_ != 0]

The magnitude of non-zero coefficients

In [38]:
final_non_zero

array([  1.36540813e+04,   1.63165294e+02,  -2.69742928e+01,
         5.20491250e+05,   4.23432580e+04,   1.17969590e+05,
        -2.73352753e+03])

In [28]:
# Checking if number of non-zero coeff. is equal to max_non_zeros value.
len(final_non_zero) is max_non_zeros

True

# Some thoughts

The model complexity decreases with shrinking number of coefficients but magnitude of non-zero weights has also impact on complexity. High magnitude of the coefficients tells us about a lot of emphasis used on that features and too small L1 penalty value used for training a model. It means that particular feature with large weight is a good predictor for the outcome and ends up overfitting to the training data. Considered range of L1 penalties values should be much wider. In that case RSS gets lower value due to error and magnitude of coefficients balance.